# Non-CivisML Model Deployment

Thanks to the flexible nature of Civis's `/services` endpoint, we can deploy much more than just CivisML models. In fact, more or less any app that lives in a Docker container and serves requests can be deployed via Civis Platform. 

As an example of a non-CivisML deployed model, here we will train a neural network to create movie recommendations using the free "MovieLens" movie rating data. This model is just for demonstration purposes and not at well tuned, but hopefully it will provide inspiration for users to write their own apps and deploy them on Civis Platform. The code that generated the Docker image that we use below ("civisanalytics/docker-scratch:muffnn_recommender-v0.0.6") can be found [here](https://github.com/civisanalytics/model-deployment/tree/muffnn_recommendation_engine). For now, a pre-built image is only accessible through Civis's private Dockerhub account, but users without access to Civis's Dockerhub can build the image themselves from the above-linked repository. 

In [1]:
from io import BytesIO, StringIO
import pickle
from zipfile import ZipFile

import civis  # Must be version 1.8.X
from muffnn import MLPRegressor
import numpy as np
import pandas as pd
import requests
from scipy import sparse

/Users/kcrum/miniconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


# Let's build a movie recommendation engine

We'll use the small MovieLens set to train an MLP. Let's get the data...

In [2]:
url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

In [3]:
response = requests.get(url)

with ZipFile(BytesIO(response.content)) as zfile:
    ratings = pd.read_csv(BytesIO(zfile.read('ml-latest-small/ratings.csv')))

In [4]:
print(ratings.shape)
ratings.head()

(100004, 4)


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


## Reshape the data into a wide, sparse format for training

In [5]:
# This cell requires a decent amount of memory. It also takes about a minute to run.
ymlp = ratings['rating']
xmlp = pd.get_dummies(ratings.drop(['rating', 'timestamp'], axis=1), columns=['userId', 'movieId'])

xtrain = sparse.csr_matrix(xmlp.values)
ytrain = ymlp.values

In [6]:
xtrain.shape

(100004, 9737)

## Build the model

We'll make a simple MLP regressor to predict movie ratings. This model has not been meaningfully tuned, but it will do for the sake of providing an example recommendation engine.

In [7]:
mlp = MLPRegressor(
    hidden_units=[200],
    n_epochs=20,
    keep_prob=0.6,
    batch_size=200,
    random_state=np.random.RandomState(3)
)
# Training takes about a minute
mlp.fit(xtrain, ytrain)

Instructions for updating:
Use the retry module or similar alternatives.


MLPRegressor(activation=<function relu at 0x1168a88c8>, batch_size=200,
       hidden_units=[200], keep_prob=0.6, n_epochs=20,
       random_state=<mtrand.RandomState object at 0x1182360d8>,
       solver=<class 'tensorflow.python.training.adam.AdamOptimizer'>,
       solver_kwargs=None, transform_layer_index=None)

## Save the model and user-item columns.

In order to deploy our model, we'll save it to the Civis /files endpoint. 

Additionally, we'll save the list of columns in the wide training data. This will enable us to reconstruct the shape of the sparse input data expected from our model.

In [8]:
with BytesIO() as bts:
    pickle.dump(mlp, bts)

    bts.seek(0)
    model_file_id = civis.io.file_to_civis(bts, 'movie_mlp.pkl', expires_at=None)

    print("Model file ID: ", model_file_id)

Model file ID:  11843062


In [9]:
with StringIO() as sio:
    # Write comma-separated column list to StringIO
    sio.write(','.join(xmlp.columns))

    sio.seek(0)
    user_item_columns_file_id = civis.io.file_to_civis(sio, 'columns.txt', expires_at=None)

    print("Column list file ID: ", user_item_columns_file_id)

Column list file ID:  11843063


## Deploy the model

To deploy a recommendation engine, we'll make the same sequence of API calls we made to deploy a standard CivisML model. However due to the data transformations necessary to build the sparse input matrix and create the ranked recommendations, we need to pass a few more environment variables. These contain the metadata required to generate the sparse input matrix. 

In [10]:
client = civis.APIClient(resources='all')

resp = client.services.post(
        name='deployed_recommender', 
        docker_image_name="civisanalytics/docker-scratch", 
        docker_image_tag="muffnn_recommender-v0.0.6", 
        cpu=1000,
        memory=8000, 
        environment_variables={'DEBUG': 1, 'MODEL_FILE_ID': model_file_id, 
                               'USER_ITEM_COLUMNS_FILE_ID': user_item_columns_file_id, 
                               'USER_PREFIX': 'userId_', 'ITEM_PREFIX': 'movieId_'})

Once we have created our service, we can start the deployment with the following API call:

In [11]:
_ = client.services.post_deployments(resp['id'])

## Let's get some recommendations!

As usual, we'll want to grab our URL and create an access token. 

In [12]:
url = client.services.get(resp['id'])['current_url']
print("URL: ", url)

token_resp = client.services.post_tokens(resp['id'], 'my_token')
print("Token: ", token_resp)

URL:  https://services--820--385354fc66b183092033d480--civis.shiny.services.civis.io
Token:  {'created_at': '2018-04-27T15:55:53.000Z', 'token': '5u4oPlI7BXoOrN-7DxESvI8Su1Y6ZtHqgyABmyH_qUr4sVhaIq1LZlEvgLMRg7q7dTYMs98HCfFnrDzyAk_E', 'name': 'my_token', 'id': 111}


Our deployed recommender will take a user ID as its input, and output one recommendation by default. 

In [13]:
import requests

# Put the token in the header of the HTTP call
headers = {"Authorization": "Bearer {}".format(token_resp['token'])}

pred_url = url + '/predict?user=10'

# Make the GET call
getresp = requests.get(pred_url, headers=headers)
print(getresp.text)

{
  "item": [
    "movieId_3030"
  ], 
  "rating": [
    4.865910053253174
  ]
}



Optionally, you can use the `ntopitems` parameter to specify the number of items you would like recommended.

In [14]:
pred_url = url + '/predict?user=10&ntopitems=5'

# Make the GET call
getresp = requests.get(pred_url, headers=headers)
print(getresp.text)

{
  "item": [
    "movieId_3030", 
    "movieId_67504", 
    "movieId_7013", 
    "movieId_83318", 
    "movieId_1217"
  ], 
  "rating": [
    4.865910053253174, 
    4.811808109283447, 
    4.804734230041504, 
    4.8038649559021, 
    4.787838935852051
  ]
}

